In [1]:
import plotly.graph_objects as go
# import kaleido
from plotly.subplots import make_subplots
import plotly.io as pio
import pandas as pd
import numpy as np
import json

from statsmodels.tsa.seasonal import seasonal_decompose

# local modules 
import sys
sys.path.insert(0, "../scripts")
import helpers as h
import seaborn as sns

In [2]:
a00, a01 = h.import_desired_data("A", "15T")
# TODO edit in function 
a00.set_index(a00["DateTime"].values, inplace=True)
a01.set_index(a01["DateTime"].values, inplace=True)
print(a00["Window Open"].unique(), a01["Window Open"].unique(), a00.columns)

[1.] [0. 1.] Index(['DateTime', 'Temp C', 'RH %', 'Room', 'Ambient Temp', 'Ambient RH',
       'Window Open'],
      dtype='object')


In [12]:
# period => # samples per unit in length of time over which seasonality occurs (here, 4 samples/1 hour) * length of time over which seasonality occurs (here, 24 hours)
n_samples = 4
seasonality_period = 24
period = n_samples*seasonality_period
print(period)

stl_obs = seasonal_decompose(a01["Temp C"],model='additive', period=period)

# difference between stl and its deriv 
stl_obs_minus_obs = h.normalize(h.normalize(stl_obs.seasonal) - h.normalize(a01["Temp C"]))



# calculate derivative of observation 
obs_deriv = h.normalize(pd.Series(np.gradient(a01["Temp C"]), a01.index, name='obs_deriv'))

stl_deriv = seasonal_decompose(obs_deriv,model='additive', period=period)

# stl_obs.plot();
# stl_deriv.plot();

96


In [15]:
fig = go.Figure()

fig.add_trace(go.Scatter(
                    x=a01["DateTime"].index,
                    y=h.normalize(a01["Window Open"]), 
                    mode='lines',
                    name="Window Sched",
                    line=dict(width=1),
                ))

fig.add_trace(go.Scatter(
                    x=a01["DateTime"].index,
                    y=h.normalize(a01["Temp C"]), 
                    mode='lines',
                    name="Obs"
                ))




fig.add_trace(go.Scatter(
                    x=stl_obs.seasonal.index,
                    y=h.normalize(stl_obs.seasonal), 
                    mode='lines',
                    name="Obs STL "
                ))


fig.add_trace(go.Scatter(
                    x=stl_obs_minus_obs.index,
                    y=stl_obs_minus_obs, 
                    mode='lines',
                    name="stl_obs_minus_obs "
                ))

                

fig.add_trace(go.Scatter(
                    x=obs_deriv.index,
                    y=obs_deriv, 
                    mode='lines',
                    name="Obs 1st Deriv "
                ))

fig.add_trace(go.Scatter(
                    x=stl_deriv.seasonal.index,
                    y=h.normalize(stl_deriv.seasonal), 
                    mode='lines',
                    name="Deriv STL"
                ))

fig.update_xaxes(
    dtick=4*60*60*1000, #1 hr in miliseconds,
    tickformat='%a-%H')